In [1]:
import pandas as pd
import json
import re

C:\Users\PJ-COMPUTERS\anaconda3\Lib\site-packages\pandas\core\computation\expressions.py:22: UserWarning: Pandas requires version '2.10.2' or newer of 'numexpr' (version '2.8.7' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\PJ-COMPUTERS\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:56: UserWarning: Pandas requires version '1.4.2' or newer of 'bottleneck' (version '1.3.7' currently installed).
  from pandas.core import (


In [2]:
# 1. Load Datasets

In [5]:
orders = pd.read_csv("C:/Users/PJ-COMPUTERS/Downloads/orders.csv")

In [7]:
with open("C:/Users/PJ-COMPUTERS/Downloads/users.json", 'r') as f:
    users = pd.DataFrame(json.load(f))

In [8]:

## Parsing SQL INSERT statements for Restaurant Master Data

In [10]:
res_data = []
with open("C:/Users/PJ-COMPUTERS/Downloads/restaurants.sql", 'r') as f:
    sql_text = f.read()
    # Regex to extract (id, 'name', 'cuisine', rating)
    matches = re.findall(r"\((\d+),\s*'([^']*)',\s*'([^']*)',\s*([\d.]+)\)", sql_text)
    for m in matches:
        res_data.append({'restaurant_id': int(m[0]), 'cuisine': m[2], 'rating': float(m[3])})
restaurants = pd.DataFrame(res_data)

In [11]:
# 2. Merge Data (Left Join)

In [12]:
df = orders.merge(users, on='user_id', how='left')
df = df.merge(restaurants, on='restaurant_id', how='left')

In [13]:
# Convert date for time-series analysis

In [15]:
df['order_date'] = pd.to_datetime(df['order_date'], format='%d-%m-%Y')

In [16]:
# 3. Export Final Dataset

In [17]:
df.to_csv('final_food_delivery_dataset.csv', index=False)
print("Final dataset created successfully with 10,000 rows.")

Final dataset created successfully with 10,000 rows.


In [18]:
gold_rev = df[df['membership'] == 'Gold'].groupby('city')['total_amount'].sum()
print(f"Top Gold Revenue City: {gold_rev.idxmax()} (₹{gold_rev.max():.2f})")

Top Gold Revenue City: Chennai (₹1080909.79)


In [19]:
aov_gold = df[df['membership'] == 'Gold']['total_amount'].mean()
print(f"Gold Member AOV: ₹{aov_gold:.2f}")

Gold Member AOV: ₹797.15


In [20]:
df['quarter'] = df['order_date'].dt.quarter
quarterly_rev = df.groupby('quarter')['total_amount'].sum()
print("\nRevenue by Quarter:\n", quarterly_rev)


Revenue by Quarter:
 quarter
1    2010626.64
2    1945348.72
3    2037385.10
4    2018263.66
Name: total_amount, dtype: float64
